# **RAG (Retrieval-Augmented Generation) Systems**

##  **1. What is a RAG System?**

- A *Retrieval-Augmented Generation (RAG)* system is a type of AI system that combines large language models (LLMs) with external information sources (like documents, databases, or knowledge bases). Instead of relying purely on the knowledge baked into the LLM, a RAG system retrieves relevant information from external sources and uses it to generate more accurate, up-to-date, and contextually relevant responses.

**Why do we need RAG?**

LLMs have limitations:

- Knowledge cutoff: They might not know recent events.
- Hallucinations: They sometimes generate false or misleading information.
- Limited memory: They can only “remember” so much context within a conversation.

RAG allows LLMs to access external knowledge, improving:

- Accuracy
- Relevance
- Coverage of specialized domains

Example use cases:

- Answering questions from large documents
- Customer support using internal knowledge bases
- Generating reports with current data

**Why not just use an LLM alone?**

- LLMs are trained on fixed datasets. If you ask about a niche domain (like proprietary company manuals or specific coding guidelines), the LLM may not know the answer.
- RAG allows dynamic, up-to-date information retrieval, combining the strengths of search engines and LLMs.


##  **2. Components of a RAG System**

A typical RAG system consists of the following components:

- **Knowledge Base:**
Where your source information is stored. Could be PDFs, text files, or a database.

- **Embeddings Generator:**
Converts documents and queries into numerical vectors that the machine can process.

- **Vector Database (Vector DB):**
Stores embeddings and allows fast similarity searches.

- **Retriever:**
Finds the most relevant documents for a given query by comparing vectors.

- **Large Language Model (LLM):**
Generates the final response using both the retrieved documents and the original query.

- **Indexer / Chunker:**
Prepares the documents for efficient storage and retrieval by splitting them into manageable pieces.


##  **3. Vector Embeddings**

**What are embeddings?**

Embeddings are numerical representations of text in a high-dimensional space.
Similar texts have similar embeddings (close in vector space), while different texts are far apart.

**Why are embeddings used?**

LLMs cannot directly search millions of documents efficiently.
Embeddings let us measure semantic similarity:
“How similar is this document to my query?”
Done via distance metrics like cosine similarity.

<p align="center">
  <img src="images/embeddings.png" width="500">
</p>

In [1]:
import os
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
aval_api_key=os.getenv("AVALAI_API_KEY")


In [2]:
client = OpenAI(    
    api_key=aval_api_key,
    base_url="https://api.avalai.ir/v1"
    )

In [3]:
res1 = client.embeddings.create(
    input="Python is a programming language.",
    model="text-embedding-3-small"
)

print(len(res1.data[0].embedding))
print(res1.data[0].embedding)

1536
[-0.011753235943615437, -0.02854209765791893, 0.005396788474172354, -0.0009712671744637191, 0.03345389664173126, -0.01746991090476513, 0.018697859719395638, 0.03110118769109249, 0.006645376328378916, 0.0072748297825455666, 0.010509807616472244, -0.010876128450036049, 0.000976426643319428, 0.0035548636224120855, 0.007847528904676437, 0.014446470886468887, -0.008879419416189194, 0.031947337090969086, -0.008384112268686295, 0.0029563670977950096, 0.004021794069558382, -0.015880798920989037, -0.038530800491571426, 0.025343237444758415, 0.036219365894794464, -0.029615264385938644, 0.03017248585820198, 0.04593977704644203, -0.020813236013054848, 0.0038463727105408907, 0.009839078411459923, -0.03735444322228432, -0.02926442213356495, 0.04957203194499016, -0.03229818120598793, -0.06290405988693237, 0.029408887028694153, -0.023671573027968407, 0.07499781996011734, -0.029883556067943573, -0.02171098068356514, -0.030255036428570747, 0.011732597835361958, 0.009870034642517567, 0.0007868167012

In [ ]:
res2 = client.embeddings.create(
    input="Bananas are yellow fruits.",
    model="text-embedding-3-small"
)

print(len(res2.data[0].embedding))
print(res2.data[0].embedding)

1536
[0.0013306529726833105, -0.07499174773693085, -0.028239157050848007, 0.003650231985375285, 0.00925054494291544, -0.017575418576598167, 0.007571994327008724, 0.061267122626304626, -0.03270706534385681, 0.03137409687042236, -0.009528246708214283, -0.015464887954294682, -0.055145345628261566, -0.0069919065572321415, 0.037347763776779175, 0.06176081299781799, -0.037619296461343765, 0.025918805971741676, 0.0074300579726696014, 0.03167031332850456, 0.0198957696557045, -0.01704470068216324, 0.006053892429918051, -0.006007608957588673, 0.048307716846466064, -0.037347763776779175, 0.026634659618139267, 0.07711461931467056, -0.013342014513909817, 0.0132185909897089, 0.03097914531826973, -0.021845851093530655, -0.10150298476219177, 0.03613822162151337, 0.004335228819400072, -0.032781120389699936, 0.023709537461400032, -7.776606071274728e-05, 0.03300328180193901, 0.0012689415598288178, 0.04154414311051369, -0.02818978764116764, 0.03633569926023483, -0.006229770369827747, -0.02008090354502201,

In [ ]:
embeddings = [res1.data[0].embedding, res2.data[0].embedding]
# Convert to numpy arrays for similarity calculations
vecs = [np.array(e) for e in embeddings]

# Example: cosine similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

similarity = cosine_similarity(vecs[0], vecs[1])
print(similarity) 


0.4978524597713467


##  **4. Vector Databases**

**What is a Vector DB?**

- A vector database is a storage system optimized for fast similarity search of embeddings.
- Instead of searching full text, we search vectors to find the most semantically relevant documents.

**Why use them?**

- Efficient for large datasets (millions of documents)
- Fast retrieval with approximate nearest neighbor (ANN) methods
- Simplifies RAG pipelines

**Popular Vector DBs:**

- ChromaDB: Lightweight, Python-friendly, ideal for small/medium projects.
- FAISS (Facebook AI Similarity Search): Optimized for large datasets, extremely fast.
- Pinecone, Weaviate, Milvus: Other options with cloud support.

```
pip install chromadb
```

In [15]:
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.Client()

# Use an embedding function
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=aval_api_key,
    api_base="https://api.avalai.ir/v1",
    model_name="text-embedding-3-small",
    )

collection = client.get_or_create_collection(
    name="my_collection",
    embedding_function=embedding_fn)

# Add documents
collection.add(
    documents=["Python is fun", "Bananas are yellow"],
    metadatas=[{"source": "doc1"}, {"source": "doc2"}],
    ids=["1", "2"],
)


##  **5. Chunking and Indexing**

**Chunking:**

- Large documents are broken into smaller pieces (chunks) so the retriever can handle them efficiently.
- Typical chunk sizes: 200–500 words.
- Helps LLM focus on relevant pieces instead of overwhelming it with a huge document.

**Indexing:**

- Organizes chunks in the vector database.
- Associates each chunk with its embedding and metadata (source, location).
- Makes retrieval fast and precise.

In [13]:
def chunk_text(text, chunk_size=100):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]


In [14]:
chunked_data = chunk_text("""A big text:
##  **3. Vector Embeddings**

**What are embeddings?**

Embeddings are numerical representations of text in a high-dimensional space.
Similar texts have similar embeddings (close in vector space), while different texts are far apart.

**Why are embeddings used?**

LLMs cannot directly search millions of documents efficiently.
Embeddings let us measure semantic similarity:
“How similar is this document to my query?”
Done via distance metrics like cosine similarity.

<p align="center">
  <img src="images/embeddings.png" width="500">
</p>
##  **4. Vector Databases**

**What is a Vector DB?**

- A vector database is a storage system optimized for fast similarity search of embeddings.
- Instead of searching full text, we search vectors to find the most semantically relevant documents.

**Why use them?**

- Efficient for large datasets (millions of documents)
- Fast retrieval with approximate nearest neighbor (ANN) methods
- Simplifies RAG pipelines

**Popular Vector DBs:**

- ChromaDB: Lightweight, Python-friendly, ideal for small/medium projects.
- FAISS (Facebook AI Similarity Search): Optimized for large datasets, extremely fast.
- Pinecone, Weaviate, Milvus: Other options with cloud support.

```
pip install chromadb
```
**5. Chunking and Indexing**

**Chunking:**

- Large documents are broken into smaller pieces (chunks) so the retriever can handle them efficiently.
- Typical chunk sizes: 200–500 words.
- Helps LLM focus on relevant pieces instead of overwhelming it with a huge document.

**Indexing:**

- Organizes chunks in the vector database.
- Associates each chunk with its embedding and metadata (source, location).
- Makes retrieval fast and precise.""")

##  **6. Retriever**

**What is a retriever?**
- A retriever is the component that searches the vector DB to find the most relevant chunks for a query.

**Types of retrievers:**
- Dense retrievers: Use embeddings for similarity search (most common in RAG)
- Sparse retrievers: Traditional keyword-based search (e.g., Elasticsearch)

In [5]:
query = "What is Python?"
results = collection.query(query_texts=[query], n_results=1)
print(results)


{'ids': [['1']], 'embeddings': None, 'documents': [['Python is fun']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'source': 'doc1'}]], 'distances': [[0.4381786584854126]]}


### ✅ Key Takeaways:

RAG systems combine retrieval with generation for more accurate, domain-specific responses.

- Core components: embeddings, vector DB, retriever, chunking, LLM.
- Vector databases make large-scale retrieval efficient.
- Chunking and indexing are crucial for performance.
-Building a RAG system involves embedding your documents, storing them in a vector DB, retrieving relevant chunks, and feeding them to an LLM for generation.